<a href="https://colab.research.google.com/github/Taiga10969/Learn-the-basics/blob/main/Transformer/what_is_the_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 言語モデル（Transformerの構造）について理解したい！
本notebookでは言語モデルの基礎中の基礎から解説していく．

## 1章　言語モデルとは
言語モデルとは，その名のお通り言語を扱うモデルであり，次の単語の確率を出力するモデルを意味します．

## 1.1 言語モデルを使ってみよう
ここでは，transformers$^\dagger$ というライブラリで提供されているモデルを使って実際に言語モデルを動かしてみることにします．<br>
今回はGPT-2モデルを用いてある単語列の次の単語を予測してみることにします<br><br>
$^\dagger$ transformers とは Hugging Face 社が公開し，最先端の NLP モデルの実装と事前学習済みモデルを提供するライブラリのことです．
<br><br>
📌プログラム中にコメントで説明を記載しましたが，こんな感じの入出力なのか！ と思ってくれればOKです！<br>
📌もっと詳しく知りたいという方は[公式ドキュメント](https://huggingface.co/docs/transformers/model_doc/gpt2#openai-gpt2)を参考にしてください！

In [ ]:
# 必要ライブラリのインポート
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# トークナイザーの定義
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',
                                            #add_bos_token = True
                                            )
 # モデルの定義
model = GPT2LMHeadModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# 入力するtextを定義
text = "Artificial intelligence (AI) is"

# 入力するtextをトークナイザーを用いて数値(id)化する
input = tokenizer(text, return_tensors='pt',)

# トークナイザーの返却値を確認
print("input　: ", input)  # keyとして 'input_ids' と 'attention_mask' がある辞書型 (正確には<class 'transformers.tokenization_utils_base.BatchEncoding'>)

# 各数値がどういう単語を示しているのかを確認
tokens = tokenizer.convert_ids_to_tokens(input.input_ids[0])
print(f"tokens : {tokens}\n","-"*100)

# モデルのgenerate関数を呼び出して入力するtextに続く単語列を生成する
output = model.generate(inputs = input['input_ids'],  #モデルへの入力は数値(id)
                          pad_token_id = 50256,
                          eos_token_id = 50256,
                          max_length=1,
                          early_stopping=True,
                          )

# モデルの出力を確認
print("output : ", output)

# モデルが出力した数値(id)を各単語に逆変換
generated_text = tokenizer.decode(output[0])
print("generated_text : ", generated_text)

input　:  {'input_ids': tensor([[ 8001,  9542,  4430,   357, 20185,     8,   318]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
tokens : ['Art', 'ificial', 'Ġintelligence', 'Ġ(', 'AI', ')', 'Ġis']
 ----------------------------------------------------------------------------------------------------
output :  tensor([[ 8001,  9542,  4430,   357, 20185,     8,   318,   257]])
generated_text :  Artificial intelligence (AI) is a


---
ここで，処理ごとにプログラムを見ていきましょう！<br>
ここでは，モデルの入出力に焦点を当ててモデル内部は考えません．<br><br>

**📌モデルへの入力**<br>
📎もちろん扱うデータはテキストですが，モデル内部では重みを掛けたりバイアスを足したりと一般的なニューラルネットワークと同じく数値計算が行われます．そこで，テキストを数値データに変換する必要があります．トークナイザーは文字列を各単語に分割 (トークン化) し，更に各単語 (トークン) を数値 (id) に変換するものです．また，トークナイザーは "attention_mask" も提供されますが一旦無視します．<br>
📎また，id 化された各トークンがどのような単語を表しているかをトークナイザーの `convert_ids_to_tokens` メソッドを使って人間が理解できるテキストの状態に逆変換してみます．
逆変換した結果を見ると，　"Artificial" という単語が "Art" と "ificial" に分割されていることがわかります．これは，トークナイザーはトークンと id がセットになったデータを用いて変換を行っています．したがって，トークナイザーが知らない単語は id に変換することができません．そこで，そのような単語が入力された場合には知っているトークンに単語を更に分割して id 化されるのです．更に，各トークンの前に "G" が表示されている場合がある．これは，この単語の前にスペースがあることを示している．これによって，このトークンが前のトークンと紐づいて１つの単語を示すものなのか，単体で１つの単語を示すものなのか区別をすることができます．<br>
📎トークナイザーが持つトークンと id のセット (ボキャブラリー) の数は，`tokenizer.vocab_size` で確認することができ，今回使用しているトークナイザーは 50257 ものボキャブラリー数があることがわかります．<br>
📎モデルへの入力は id を入力します．<br><br>


**📌モデルの出力**<br>
📎モデルの出力を見ると，これまた id ということがわかります．8001から318はモデルに入力した id ですが，それに続く id である 257 はモデルが生成したものになります． これらの id　をトークナイザーの `decode` メソッドを使って単語に戻すと，人間が理解できるテキストとなって入力したテキストの次の単語の予測結果を確認することができます．<br>
📎しかし，これではあるテキストの次の単語を予測したに過ぎません．今回でいうと，"Artificial intelligence (AI) is" の後に続くの単語として "a" を予測しただけです．モデルは次に "Artificial intelligence (AI) is a" に続く単語を予測します．このように，入力として与えた文章+これまで生成した単語から次の単語としてふさわしい単語の予測を繰り返すのが，言語モデルにおける言語生成のプロセスになります．<br>
📎 `generate()` の引数の `max_length` の値を変えて実行してみましょう！ また他の引数も変化させてみたりして，挙動を確認してみましょう！ `max_length` はモデルが生成する単語の数を指定するものです．16などの数字にすると最大で16単語モデルが新しい次単語を予測することになり，長い文章を生成できるようになります．

---